# Running WPS from notebook using OWSlib
Error at the end - OWSlib is complaining about UTF-8


In [1]:
from owslib.wps import WebProcessingService, monitorExecution, printInputOutput
from os import system
import time
from owslib.etree import etree

In [2]:
wps_url = "http://localhost:8080/geoserver/wps"
# wps_url= "http://earss1.ornl.gov:80/geoserver/wps"
# wps_url = "http://localhost:8093/wps"
# wps_url = "https://publicwiki.deltares.nl/display/OET/WPS"
wps = WebProcessingService(url=wps_url, verbose=False)

In [3]:
print (wps.identification.title + ':' + wps.version)

Prototype GeoServer WPS:1.0.0


In [4]:
# for process in wps.processes:
#     print ('%s : \t %s' % (process.identifier, process.abstract))

In [41]:
p = wps.describeprocess(identifier='JTS:length')
print("Inputs")
for input in p.dataInputs:
   printInputOutput(input)
   print ('\n')
print("Outputs")
for output in p.processOutputs:
   printInputOutput(output)
   print ('\n')

Inputs
 identifier=geom, title=geom, abstract=Input geometry, data type=ComplexData
 Supported Value: mimeType=text/xml; subtype=gml/3.1.1, encoding=None, schema=None
 Supported Value: mimeType=text/xml; subtype=gml/2.1.2, encoding=None, schema=None
 Supported Value: mimeType=application/wkt, encoding=None, schema=None
 Supported Value: mimeType=application/json, encoding=None, schema=None
 Supported Value: mimeType=application/gml-3.1.1, encoding=None, schema=None
 Supported Value: mimeType=application/gml-2.1.2, encoding=None, schema=None
 Default Value: mimeType=text/xml; subtype=gml/3.1.1, encoding=None, schema=None 
 minOccurs=1, maxOccurs=1


Outputs
 identifier=result, title=result, abstract=None, data type=LiteralData
 Default Value: None 
 reference=None, mimeType=None




In [33]:
def wrapXML(proces,geom):
    xml='<?xml version="1.0" encoding="UTF-8"?><wps:Execute version="1.0.0" service="WPS" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.opengis.net/wps/1.0.0" xmlns:wfs="http://www.opengis.net/wfs" xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:gml="http://www.opengis.net/gml" xmlns:ogc="http://www.opengis.net/ogc" xmlns:wcs="http://www.opengis.net/wcs/1.1.1" xmlns:xlink="http://www.w3.org/1999/xlink" xsi:schemaLocation="http://www.opengis.net/wps/1.0.0 http://schemas.opengis.net/wps/1.0.0/wpsAll.xsd">  <ows:Identifier>%s</ows:Identifier><wps:DataInputs><wps:Input><ows:Identifier>geom</ows:Identifier><wps:Data><wps:ComplexData mimeType="application/json"><![CDATA[%s]]></wps:ComplexData></wps:Data></wps:Input></wps:DataInputs><wps:ResponseForm><wps:RawDataOutput><ows:Identifier>result</ows:Identifier></wps:RawDataOutput></wps:ResponseForm></wps:Execute>'%(proces,geom)
    return xml.strip()

In [43]:
from osgeo import ogr
import requests
poly = ogr.Geometry(ogr.wkbLinearRing)
polygon = [(-102.8184, 39.5273), (-102.8184, 37.418), (-101.2363, 37.418), (-101.2363, 39.5273), (-102.8184, 39.5273)]
for i in polygon:    
    poly.AddPoint(*i)

p=poly.ExportToJson()
dowyslania=wrapXML("JTS:length",p)
# ExportToWkt()


r = requests.post("http://185.52.194.216:8080/geoserver/wps?JTS:numGeometries", data=dowyslania)
print(r.status_code, r.reason)
print(r.text[:])

200 OK
7.382799999999989


In [38]:
from owslib.wps import GMLMultiPolygonFeatureCollection
from owslib.wps import WebProcessingService, monitorExecution, printInputOutput
from os import system
import time
from owslib.etree import etree
wps_url = "http://localhost:8080/geoserver/wps"
wps = WebProcessingService(url=wps_url, verbose=False)

polygon = [(-102.8184, 39.5273), (-102.8184, 37.418), (-101.2363, 37.418), (-101.2363, 39.5273), (-102.8184, 39.5273)]
featureCollection = GMLMultiPolygonFeatureCollection( [polygon] )
print( featureCollection.getXml())
    

inputs1 = [ ("geom",featureCollection)]



# execute = wps.execute(identifier='JTS:numGeometries', inputs=inputs1)

<Element {http://www.opengis.net/wps/1.0.0}Data at 0x7fb9ef757dc8>
